# N07. Conversión a Excel
En este documento se explicará el proceso de conversión a un libro de Excel, de los datos de entrada y salida de los siguientes procesos:

* Proyección de precios mensuales de energía en bolsa.
* Selección de la alternativa.
* Presupuesto de la compra de energía.

## 7.1. Paquetes de conversión de datos DataFrame a Excel
Hay tres paquetes que se utilizarán en la conversión de datos en formato DataFrame a datos en hojas Excel:

1. **xlswriter**. No requiere tener instalado Excel. Solo permite escribir datos en DataFrame a Excel.
2. **openpyxl**. No requiere tener instalado Excel. Permite escribir datos en DataFrame a Excel, pero también permite leer y editar.
3. **xlwings***. Requiere tener instalado Excel. Permite escribir datos que están en DataFrame a Excel, pero también permite leer y editar. Adicionalmente, permite correr funciones y subrutinas VBA desde Python.

## 7.2. Creación del libro Excel y hojas iniciales con 'xlswriter'



In [107]:
#*******************************************************************************
# 7.1. Carga los paquetes requeridos.
#*******************************************************************************
import numpy as np
import pandas as pd
import xlsxwriter
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter
import xlwings as xw
%matplotlib inline

In [244]:
#*******************************************************************************
# 7.2. Lee variables para realizar proyección.
#*******************************************************************************
aportes_m = pd.read_parquet('parquet/aportes_m.parquet.gzip', \
    engine='fastparquet')
pbhL = pd.read_parquet('parquet/pbhL.parquet.gzip', \
    engine='fastparquet')
pbm = pd.read_parquet('parquet/pbm.parquet.gzip', \
    engine='fastparquet')

In [245]:
#*******************************************************************************
# 7.3. Acondiona las variables para Excel.
#*******************************************************************************
pbm = pbm.reset_index()
pbm.precio = pbm.precio.round(2)
pbm.preEsca = pbm.preEsca.round(2)
pbm.preConEsca = pbm.preConEsca.round(2)
#pbm.mes = pd.to_datetime(pbm.mes).astype(str)
# pbm.mes = '01/' + pbm.mes.to_period('M').strftime('%m/%Y')

In [238]:
(pbm.mes[0])

Timestamp('2000-01-01 00:00:00')

In [286]:
#*******************************************************************************
# 7.4. Crea libro y hojas de 'EnergiaCaracara_22_28'.
#*******************************************************************************
nombreHojas = ['h01_portada', 'h02_bases_calculo']
workbook = xlsxwriter.Workbook('xlsx_salida/EnergiaCaracara_22_28.xlsx')
for i, nombre in enumerate(nombreHojas, start=1):
    worksheet = workbook.add_worksheet(nombre)
workbook.close()

In [250]:
#*******************************************************************************
# 7.5. Carga imagen de la portada.
#*******************************************************************************
libro = load_workbook('xlsx_salida/EnergiaCaracara_22_28.xlsx')
hoja = libro['h01_portada']
imagen = Image('jpg/portadaCepsa.jpg')
hoja.add_image(imagen, 'A12')
libro.save('xlsx_salida/EnergiaCaracara_22_28.xlsx')

In [287]:
#*******************************************************************************
# 7.6. Carga el data frame pbh en la hoja 'h03_pbh'.
#*******************************************************************************
escritor = pd.ExcelWriter('xlsx_salida/EnergiaCaracara_22_28.xlsx', 
    engine='openpyxl', mode='a')
pbm.to_excel(escritor, sheet_name='h04_pbm', startrow=5, startcol=0, 
    header=True, index=False)
escritor.save()
#xlsxwriter no agrega o edita datos. Ahora se trabaja con openpyxl.
libro = load_workbook('xlsx_salida/EnergiaCaracara_22_28.xlsx')
hoja = libro['h04_pbm']
# encabezados = pbm.columns
tab = Table(displayName="Tabla1", ref='A6:D277')
# Add a default style with striped rows and banded columns
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
tab.tableStyleInfo = style
#
for i, _ in enumerate(pbm.columns, start=1):
    letra = get_column_letter(i)
    hoja.column_dimensions[letra].width = 15
#
hoja.add_table(tab)
libro.save('xlsx_salida/EnergiaCaracara_22_28.xlsx')
#Incluye la fórmula.
hoja.cell(7, 4).value = "=IF(B7 < C7, B7, C7)"
# "=IF(hoja.cell(7, 2).value < hoja.cell(7, 3).value; hoja.cell(7, 2).value; hoja.cell(7, 3).value)"
#
libro.save('xlsx_salida/EnergiaCaracara_22_28.xlsx')

In [188]:
pbm.reset_index()

,mes,precio,preEsca,preConEsca
0,01/01/2000,36.77,2500.00,36.77
1,01/02/2000,40.26,2500.00,40.26
2,01/03/2000,37.63,2500.00,37.63
3,01/04/2000,44.29,2500.00,44.29
4,01/05/2000,37.29,2500.00,37.29
...,...,...,...,...
266,01/03/2022,227.00,854.16,227.00
267,01/04/2022,143.79,915.62,143.46
268,01/05/2022,105.29,971.18,105.29
269,01/06/2022,104.03,952.52,104.03


## 7.3. Edición del libro Excel con 'xlwings'